#Algoritmo Genético
Copyright (C) 2018, Murilo Pagliuso

In [0]:
import random as r

BASE = '010010010010'

class Populacao:
    MAX_POPULACAO = 8
    TAMANHO_TORNEIO = 5
    TAXA_UNIFORME = 0.5
    TAXA_MUTACAO = 0.015
    elitismo = True

    def __init__(self, solucao, gerar=True, cromossomos=None):
        self.solucao = solucao
        if gerar == True:
            self.cromossomos = self._gerar()
        else:
            if cromossomos == None:
                self.cromossomos = []
            else:
                self.cromossomos = cromossomos

        for c in self.cromossomos:
            c.calcularaptidao(self.solucao)

    def setsolucao(self, solucao):
        self.solucao = solucao

    def _gerar(self):
        return [Cromossomo() for cromossomo in range(0, self.MAX_POPULACAO)]

    def setcromossomo(self, index, cromo):
        self.cromossomos[index] = cromo

    def getmelhor(self):
        c1 = self.cromossomos[0]
        for c in self.cromossomos:
            if c.aptidao > c1.aptidao:
                c1 = c
        return c1

    def getpior(self):
        return self.cromossomos[self.getindicepior()]

    def getindicepior(self):
        indice = 0
        c1 = self.cromossomos[0]
        for i in range(0, len(self.cromossomos)):
            if self.cromossomos[i].aptidao < c1.aptidao:
                c1 = self.cromossomos[i]
                indice = i
        return indice

    def __str__(self):
        return self.cromossomos

    def mutacao(self, cromo):
        for i in range(0, len(cromo.genes)):
            if r.random() <= self.TAXA_MUTACAO:
                gene = r.choice([0, 1])
                cromo.setgene(i, gene)

    def selecaoroleta(self):
        soma = 0
        porc = 0
        for cromo in self.cromossomos:
            soma += cromo.aptidao
        rnd = float(r.random() * soma)

        for cromo in self.cromossomos:
            if (rnd >= porc) and (rnd <= (porc + cromo.aptidao)):
                return cromo
            porc += cromo.aptidao
        return None

    def evoluir(self, pop):
        newPop = Populacao(pop.solucao, True)

        offset_elitismo = 0
        if pop.elitismo:
            pior = newPop.getindicepior()
            newPop.cromossomos[pior] = pop.getmelhor()
            offset_elitismo = 1
        else:
            offset_elitismo = 0

        for i in range(offset_elitismo, len(pop.cromossomos)):
            cromo1 = pop.selecaoroleta()
            cromo2 = pop.selecaoroleta()

            newCromo = self.crossover(cromo1, cromo2)
            newCromo.calcularaptidao(pop.solucao)

            newPop.cromossomos[newPop.getindicepior()] = newCromo

        for i in range(offset_elitismo, len(newPop.cromossomos)):
            self.mutacao(newPop.cromossomos[i])
            newPop.cromossomos[i].calcularaptidao(pop.solucao)
        return newPop

    def crossover(self, cromo1, cromo2):
        newCromo = cromo1
        for i in range(0, len(cromo1)):
            if r.random() <= self.TAXA_UNIFORME:
                newCromo.setgene(i, cromo1.genes[i])
            else:
                newCromo.setgene(i, cromo2.genes[i])
        return newCromo


class Cromossomo:
    MAX_GENES = 12
    aptidao = 0
    porcao = 0
    intervalo = []

    def __init__(self, genes=None):
        self.genes = genes or self._gerar()

    def _gerar(self):
        cromo = []
        for i in range(0, self.MAX_GENES):
            cromo.append(r.choice([0, 1]))
        return ''.join(map(str, cromo))

    def calcularaptidao(self, solucao):
        apt = 0
        for i in range(0, self.MAX_GENES):
            if self.genes[i] == solucao[i]:
                apt += 1
        self.aptidao = apt

    def setporcao(self, porc):
        self.porcao = porc

    def calcularintervalo(self, comeco):
        self.intervalo = [comeco, comeco + self.porcao]

    def setgene(self, index, gene):
        s = ''
        for i in range(len(self.genes)):
            if i == index:
                s += str(gene)
            else:
                s += self.genes[i]
        self.genes = s

    def __str__(self):
        return self.genes

    def __len__(self):
        return len(self.genes)

In [6]:
pop = Populacao(BASE, True)
geracoes = 0
geracoes_max = 100

melhor = None
while pop.getmelhor().aptidao < len(BASE) and geracoes < geracoes_max:
    geracoes += 1
    pop = pop.evoluir(pop)
    melhor = pop.getmelhor()
    print('GERACAO ' + str(geracoes) + ', MELHOR: ' + str(melhor) +
          ', APTIDAO: ' + str(melhor.aptidao))

print('')
if melhor.aptidao < len(BASE):
    print('MELHOR SOLUCAO: ' + str(melhor))
else:
    print('SOLUCAO ENCONTRADA EM ' + str(geracoes) + ' GERACOES: ' +
          str(melhor))

print('APTIDAO: ' + str(melhor.aptidao))

GERACAO 1, MELHOR: 010010000101, APTIDAO: 8
GERACAO 2, MELHOR: 010010011001, APTIDAO: 9
GERACAO 3, MELHOR: 101010010010, APTIDAO: 9
GERACAO 4, MELHOR: 010101001010, APTIDAO: 7
GERACAO 5, MELHOR: 011000110011, APTIDAO: 8
GERACAO 6, MELHOR: 001010000010, APTIDAO: 9
GERACAO 7, MELHOR: 001010000010, APTIDAO: 9
GERACAO 8, MELHOR: 000110010010, APTIDAO: 10
GERACAO 9, MELHOR: 111001110010, APTIDAO: 7
GERACAO 10, MELHOR: 000000110000, APTIDAO: 8
GERACAO 11, MELHOR: 010011110110, APTIDAO: 9
GERACAO 12, MELHOR: 011010010111, APTIDAO: 9
GERACAO 13, MELHOR: 010110010000, APTIDAO: 10
GERACAO 14, MELHOR: 010110010000, APTIDAO: 10
GERACAO 15, MELHOR: 000010010000, APTIDAO: 10
GERACAO 16, MELHOR: 011010010000, APTIDAO: 10
GERACAO 17, MELHOR: 011010010010, APTIDAO: 11
GERACAO 18, MELHOR: 011010110010, APTIDAO: 10
GERACAO 19, MELHOR: 011010110010, APTIDAO: 10
GERACAO 20, MELHOR: 011011110010, APTIDAO: 9
GERACAO 21, MELHOR: 011011110010, APTIDAO: 9
GERACAO 22, MELHOR: 110010010010, APTIDAO: 11
GERACAO 23